In [31]:
#Import The Necessary Library 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import os
import re
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
from keras.callbacks import Callback
np.random.seed(42)

In [32]:
#Load The Datasets
male_data = pd.read_csv("/content/drive/MyDrive/project/Dataset/Classify_Nationality_Male.csv")
female_data = pd.read_csv("/content/drive/MyDrive/project/Dataset/Classify_Nationality_Females.csv")

In [33]:
#First Fuve Rows of dataset 
male_data.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


**Perfrom the Pre-Process**

In [34]:
#Replace List (i,e) the sir name that need to replace 
repl_list = ['s/o','d/o','w/o','/','&',',','-']

In [35]:
#Remove the unnecessary punct,
def clean_data(name):
	name = str(name).lower()
	name = (''.join(i for i in name if ord(i)<128)).strip()
	for repl in repl_list:
		name = name.replace(repl," ")
	if '@' in name:
		pos = name.find('@')
		name = name[:pos].strip()
	name = name.split(" ")
	name = " ".join([each.strip() for each in name])
	return name

In [36]:
def remove_records(merged_data):
	merged_data['delete'] = 0
	merged_data.loc[merged_data['name'].str.find('with') != -1,'delete'] = 1	
	merged_data.loc[merged_data['count_words']>=5,'delete']=1
	merged_data.loc[merged_data['count_words']==0,'delete']=1
	merged_data.loc[merged_data['name'].str.contains(r'\d') == True,'delete']=1
	cleaned_data = merged_data[merged_data.delete==0]
	return cleaned_data

In [37]:
#Concatinate both the dataset 
merged_data = pd.concat((male_data,female_data),axis=0)

In [38]:
merged_data.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [39]:
merged_data['name'] = merged_data['name'].apply(clean_data)
merged_data['count_words'] = merged_data['name'].str.split().apply(len)

In [40]:
cleaned_data = remove_records(merged_data)

In [41]:
cleaned_data.head()

,name,gender,race,count_words,delete
0,barjraj,m,indian,1,0
1,ramdin verma,m,indian,2,0
2,sharat chandran,m,indian,2,0
3,birender mandal,m,indian,2,0
4,amit,m,indian,1,0


In [71]:
cleaned_data.shape

(29868, 5)

In [42]:
#Drop the Dupliactes Value 
indian_cleaned_data = cleaned_data[['name','count_words']].drop_duplicates(subset='name',keep='first')
indian_cleaned_data['label'] = 'indian'

In [43]:
indian_cleaned_data.head()

,name,count_words,label
0,barjraj,1,indian
1,ramdin verma,2,indian
2,sharat chandran,2,indian
3,birender mandal,2,indian
4,amit,1,indian


In [44]:
len(indian_cleaned_data)

13754

In [45]:
#Now ADD the Non-indans With the help of faker Function 
pip install Faker

In [46]:
from faker import Faker
fake=Faker()

fake.name()

'Joshua Reed'

In [47]:
#Here get the fake name around 14k coz the indian names around 13.75k
column_name=["name"]
fake = Faker(['en_US'])
data = [fake.name() for i in range(0,14000)]
data = pd.DataFrame(data,columns=column_name)
data.head()

,name
0,Jeffrey Castaneda
1,Caroline Gordon
2,Daniel White
3,Lori Rojas
4,Mr. Ralph Tyler


In [48]:
data['count_words'] = data['name'].str.split().apply(len)

In [49]:
data.head()

,name,count_words
0,Jeffrey Castaneda,2
1,Caroline Gordon,2
2,Daniel White,2
3,Lori Rojas,2
4,Mr. Ralph Tyler,3


In [50]:
cleaned_data1 = remove_records(data)

In [51]:
cleaned_data1.head()

,name,count_words,delete
0,Jeffrey Castaneda,2,0
1,Caroline Gordon,2,0
2,Daniel White,2,0
3,Lori Rojas,2,0
4,Mr. Ralph Tyler,3,0


In [72]:
cleaned_data1.shape

(14000, 3)

In [52]:
non_indian_data = cleaned_data1[['name','count_words']].drop_duplicates(subset='name',keep='first')
non_indian_data['label'] = 'non_indian'

In [53]:
non_indian_data.head()

,name,count_words,label
0,Jeffrey Castaneda,2,non_indian
1,Caroline Gordon,2,non_indian
2,Daniel White,2,non_indian
3,Lori Rojas,2,non_indian
4,Mr. Ralph Tyler,3,non_indian


In [54]:
len(non_indian_data)

12872

In [55]:
indian_cleaned_data['count_words'].value_counts()

2    7954
1    4322
3    1344
4     134
Name: count_words, dtype: int64

In [56]:
non_indian_data['count_words'].value_counts()
#Here is the disbalanaced on the count_words on Non-Indian

2    12301
3      489
4       82
Name: count_words, dtype: int64

In [57]:
#Let's make a balance between the count words 
#Count_Words=1
two_word_names = non_indian_data[non_indian_data['count_words']==2]['name']
one_word_req = 3000
names_one_two_words = [each.split()[0] for each in two_word_names[:one_word_req]] + list(two_word_names[one_word_req:])
count_words = [1] * one_word_req + [2] * len(two_word_names[one_word_req:])
not_two_words_pd  = non_indian_data[non_indian_data['count_words']!=2]
one_two_words_pd = pd.DataFrame({'name':names_one_two_words,'count_words':count_words})
non_indian_data = pd.concat((not_two_words_pd,one_two_words_pd),axis=0)
non_indian_data['count_words'].value_counts()
non_indian_data['label'] = 'non_indian'
non_indian_data = non_indian_data[non_indian_data['count_words']<5]
non_indian_data['count_words'].value_counts()

2    9301
1    3000
3     489
4      82
Name: count_words, dtype: int64

In [58]:
#Count_Words=4
two_word_names = non_indian_data[non_indian_data['count_words']==2]['name']
one_word_req = 1500
names_one_two_words = [each.split()[0] for each in two_word_names[:one_word_req]] + list(two_word_names[one_word_req:])
count_words = [4] * one_word_req + [2] * len(two_word_names[one_word_req:])
not_two_words_pd  = non_indian_data[non_indian_data['count_words']!=2]
one_two_words_pd = pd.DataFrame({'name':names_one_two_words,'count_words':count_words})
non_indian_data = pd.concat((not_two_words_pd,one_two_words_pd),axis=0)
non_indian_data['count_words'].value_counts()
non_indian_data['label'] = 'non_indian'
non_indian_data = non_indian_data[non_indian_data['count_words']<5]
non_indian_data['count_words'].value_counts()

2    7801
1    3000
4    1582
3     489
Name: count_words, dtype: int64

In [59]:
#Count_Words=3
two_word_names = non_indian_data[non_indian_data['count_words']==2]['name']
one_word_req = 2000
names_one_two_words = [each.split()[0] for each in two_word_names[:one_word_req]] + list(two_word_names[one_word_req:])
count_words = [3] * one_word_req + [2] * len(two_word_names[one_word_req:])
not_two_words_pd  = non_indian_data[non_indian_data['count_words']!=2]
one_two_words_pd = pd.DataFrame({'name':names_one_two_words,'count_words':count_words})
non_indian_data = pd.concat((not_two_words_pd,one_two_words_pd),axis=0)
non_indian_data['count_words'].value_counts()
non_indian_data['label'] = 'non_indian'
non_indian_data = non_indian_data[non_indian_data['count_words']<5]
non_indian_data['count_words'].value_counts()

2    5801
1    3000
3    2489
4    1582
Name: count_words, dtype: int64

In [60]:
#Now concate non_indian/indian then save it to .csv file 
full_data = pd.concat((non_indian_data[['name','label']],indian_cleaned_data[['name','label']]),axis=0)
full_data = full_data.sample(frac=1)
full_data.to_csv("name_data.csv",index=False)
from google.colab import files
files.download('name_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [61]:
#Now load it again 
data_url = "/content/name_data (2).csv"
name_data = pd.read_csv(data_url)

In [62]:
name_data.head()

,name,label
0,hitesh widhuri,indian
1,ravinder,indian
2,sarip ahmad,indian
3,Loretta Singleton,non_indian
4,ram awtar,indian


In [63]:
name_data['label'].value_counts()

indian        13754
non_indian    12872
Name: label, dtype: int64

In [64]:
#Train Test Split 
from sklearn.model_selection import train_test_split
X = name_data['name'].astype(str)
Y = name_data['label']
train_names,test_names,train_labels,test_labels = train_test_split(X,Y,test_size=0.2,random_state =42,stratify=Y)

In [65]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report
vectorizer = CountVectorizer()
X_ = vectorizer.fit_transform(train_names.values.astype('U'))
len(vectorizer.get_feature_names())

8646

In [66]:
model = MultinomialNB()
model.fit(X_,train_labels)

X_test = vectorizer.transform(test_names.values.astype('U'))

test_predicted = model.predict(X_test)

print(classification_report(test_labels,test_predicted))

              precision    recall  f1-score   support

      indian       1.00      0.99      1.00      2751
  non_indian       0.99      1.00      1.00      2575

    accuracy                           1.00      5326
   macro avg       1.00      1.00      1.00      5326
weighted avg       1.00      1.00      1.00      5326



In [67]:
#Test th model on the new data 
check_new_names = ['lalitha','tyson','shailaja','shyamala','vishwanathan','ramanujam','conan','kryslovsky',
'ratnani','diego','kakoli','shreyas','brayden','shanon']
X_new = vectorizer.transform(check_new_names)
predictions_nb_cv = model.predict(X_new)
test = pd.DataFrame({'names':check_new_names,'predictions_nb_cv':predictions_nb_cv}) 
test

,names,predictions_nb_cv
0,lalitha,indian
1,tyson,indian
2,shailaja,indian
3,shyamala,indian
4,vishwanathan,indian
5,ramanujam,indian
6,conan,indian
7,kryslovsky,indian
8,ratnani,indian
9,diego,indian
